In [ ]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from sklearn.linear_model import LinearRegression
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt
from collections import Counter
import json
import typing as tp
import ast
from datetime import datetime
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [ ]:
tags_helper_table = lib.find_tables_in_hahn_folder('//home/cloud_analytics/dwh/raw/crm/tag_bean_rel')[-1]
tags_table = lib.find_tables_in_hahn_folder('//home/cloud_analytics/dwh/raw/crm/tags')[-1]

In [ ]:
tags_table

# 1. Добавление

In [ ]:
req = """
select DISTINCT
    lead_id as task_id,
    first_name, 
    last_name,
    account_name,
    phone_mobile,
    email,
    status,
    lead_source, 
    coalesce(lead_source_description, '') as lead_source_description, 
    '' as description, 
    timezone
from "//home/cloud_analytics/kulaga/leads_cube"
where 
    lead_source = 'Marketo'
    and status in ('Assigned', 'New')
    AND lower(tag_name) == 'прозвон_кц'
    AND user_name == 'admin'
    AND phone_mobile != ''
    AND isNotNull(phone_mobile)
FORMAT TabSeparatedWithNames
""".encode('utf-8')
df = lib.execute_query(req)

In [ ]:
df['phone_mobile'] = df['phone_mobile'].astype(str)

In [ ]:
old_df = lib.execute_query("""
SELECT 
    DISTINCT
    task_id
FROM "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output/marketo_source_drop_task_id"
FORMAT TabSeparatedWithNames
""")

In [ ]:
df = df[~df["task_id"].isin(old_df["task_id"])]

In [ ]:
df.shape

In [ ]:
str(datetime.now()).replace(" ", "T")

In [ ]:
if len(df) > 0:
    lib.save_table(str(datetime.now()).replace(" ", "T"), 
               "//home/cloud_analytics/call_center/input/marketo", df)

## dropped list update

In [ ]:
added_df = pd.DataFrame(df["task_id"])

In [ ]:
lib.save_table("marketo_source_drop_task_id", 
               "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output", added_df, append=True)

# 2. Update Lead

In [ ]:
req = """
SELECT
    toInt64(NOW()) as Timestamp,
    CAST(task_id as String) as CRM_Lead_ID,
    relevant_contact_name as First_name,
    lower(relevant_contact_email) as Email_new,
    lower(email) as email,
    relevant_contact_phone as Phone_2,
    CAST(Dimension as String) as Dimensions,
    if (result = 'decline', 'Не готов общаться', '')  as Dimension_1,
    if (status = 'no_response', 'Не дозвонились', '')  as Dimension_2,
    if (((is_business or has_interest or has_tech_issues) and manager_required) OR
        (is_business and has_interest), '["validated"]', '["annulated"]') as Tags,
    if (Tags = '["annulated"]', 'Recycled', 'Assigned') as Status,
    toDateTime(toInt64(assignment_approve_time / 1000)) as Last_communication_date,
    Comment as Notes, 
    'Представляет юр. лицо: ' || 
    toString(ifNull(is_business, 0)) || 
    ', Интерес: ' || 
    toString(ifNull(has_interest, 0))  || 
    ', Сложные тех. вопросы: ' || 
    toString(ifNull(has_tech_issues, 0)) || 
    ', Требуется менеджер: ' || 
    toString(ifNull(manager_required, 0)) as Description
FROM "//home/cloud_analytics/call_center/output/marketo_output" as a
ANY LEFT JOIN (
    SELECT
        max(email) as email,
        lead_id
    FROM "//home/cloud_analytics/kulaga/leads_cube"
    GROUP BY lead_id
) as b
ON CRM_Lead_ID == lead_id
FORMAT TabSeparatedWithNames
""".encode('utf-8')

update_df = lib.execute_query(req)

In [ ]:
update_df[update_df['CRM_Lead_ID'] == '59d45f08-91ba-11ea-b22f-dbe6e205e325']

In [ ]:
def with_mails(x):
    add = []
    if not pd.isnull(x['email']):
        add.append(lib.works_with_emails(x['email'].strip()))
    if not pd.isnull(x['Email_new']):
        add.append(lib.works_with_emails(x['Email_new'].strip()))
    return list(set(add))

In [ ]:
update_df['Email'] = update_df[['email', "Email_new"]].apply(lambda x: with_mails(x), axis=1)

In [ ]:
update_df.drop(columns=['email', "Email_new"], inplace=True)

In [ ]:
def array_helper(x):
    return ast.literal_eval(x.replace("\u00016", '"').replace("\u0001r", '"').replace(";", '", '))

In [ ]:
update_df['Dimensions'] = update_df['Dimensions'].apply(lambda x: array_helper(x))

In [ ]:
def add_dims(x):
    added = []
    if x['Dimension_1'] != '':
        added.append(x['Dimension_1'])
    if x['Dimension_2'] != '':
        added.append(x['Dimension_2'])
    return x['Dimensions'] + added

In [ ]:
update_df['Dimensions'] = update_df[['Dimensions', 'Dimension_1', 'Dimension_2']].apply(lambda x:
    add_dims(x), axis=1)

In [ ]:
update_df.drop(columns = ['Dimension_1', 'Dimension_2'], inplace=True)

In [ ]:
update_df['Tags'] = update_df['Tags'].astype(str)

In [ ]:
update_df['Dimensions'] = update_df['Dimensions'].astype(str)
update_df['Email'] = update_df['Email'].astype(str)

In [ ]:
update_df['Dimensions'] = update_df['Dimensions'].apply(lambda x:
                                                        x.replace("'", '"') if not pd.isnull(x) else np.nan)
update_df['Tags'] = update_df['Tags'].apply(lambda x: x.replace("'", '"') if not pd.isnull(x) else np.nan)
update_df['Email'] = update_df['Email'].apply(lambda x: x.replace("'", '"') if not pd.isnull(x) else np.nan)

In [ ]:
old_df = lib.execute_query("""
SELECT
    DISTINCT 
    CRM_Lead_ID || '__' || toString(ifNull(Last_communication_date, '')) as key
FROM "//home/cloud_analytics/export/crm/update_call_center_leads/update_leads"
FORMAT TabSeparatedWithNames
""")

In [ ]:
update_df = update_df[~update_df[['CRM_Lead_ID', 'Last_communication_date']]\
          .apply(lambda x: x['CRM_Lead_ID'] + "__" + x['Last_communication_date'], axis=1)\
          .isin(old_df['key'])]

In [ ]:
update_df.shape

In [ ]:
update_df[update_df['CRM_Lead_ID'] == '59d45f08-91ba-11ea-b22f-dbe6e205e325']

In [ ]:
lib.save_table('update_leads', "//home/cloud_analytics/export/crm/update_call_center_leads",
               update_df, append=True)